In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

from lazypredict.Supervised import LazyClassifier
from pyrdf2vec import RDF2VecTransformer
from pyrdf2vec.embedders import Word2Vec, FastText
from pyrdf2vec.graphs import KG
from pyrdf2vec.samplers import WideSampler, UniformSampler
from pyrdf2vec.walkers import HALKWalker, NGramWalker, CommunityWalker, RandomWalker, WalkletWalker, Walker, WLWalker

pd.options.display.float_format = '{:,.4f}'.format
RANDOM_STATE = 22
PATH = "ENTERPATH HERE"

In [2]:
df = pd.read_csv(PATH, sep= '\t')
df.head()
df = df.rename(columns={"Unnamed: 0":"uri", "0":"label"})

train_size= int(len(df)*0.2)
test_data = df.iloc[:train_size]
train_data = df.iloc[train_size:]

train_entities = [entity for entity in train_data['uri']]
train_labels = list(train_data["label"])

test_entities = [entity for entity in test_data['uri']]
test_labels = list(test_data["label"])


entities = train_entities+test_entities
labels = train_labels + test_labels

In [3]:
kg = KG(
    location = "ENTER PATH HERE",
    skip_predicates={"http://biaslens.com/bias"},
    skip_verify = True
    )


In [4]:
## Halk Walks
rdf2vec = RDF2VecTransformer(Word2Vec(workers=2, epochs=20), 
        walkers=[
        HALKWalker(
            2,
            None,
            n_jobs=4,
            sampler=UniformSampler(),
            random_state=RANDOM_STATE,
            md5_bytes=None,
        )
    ],
verbose=1
)
embeddings, literals = rdf2vec.fit_transform(kg,  np.array(entities))

100%|██████████| 37554/37554 [00:12<00:00, 2912.79it/s]


Extracted 354105 walks for 37554 entities (66.6287s)
Fitted 354105 walks (35.7364s)


In [5]:
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, f1_score
def get_scores(X_train, X_test, y_train, y_test):
    arr = []
    clfs = ["XGBClassifier", "LogisticRegression", "RandomForestClassifier", "SVC", "KNeighborsClassifier"]
    for clf in clfs:
        model = eval(clf+"()")
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        arr.append((clf, accuracy_score(y_test, preds), f1_score(y_test, preds, average="macro")))
    return pd.DataFrame(arr, columns=["Model", "Accuracy", "F1 Macro"])

In [6]:
train_embeddings = np.array(embeddings[:len(train_entities)])
test_embeddings = np.array(embeddings[len(train_entities):])

scores = get_scores(train_embeddings, test_embeddings, train_labels, test_labels)
scores
# clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
# models,predictions = clf.fit(train_embeddings, test_embeddings, train_labels, test_labels)
# models

,Model,Accuracy,F1 Macro
0,XGBClassifier,0.9583,0.9576
1,LogisticRegression,0.9176,0.9167
2,RandomForestClassifier,0.9450,0.9442
3,SVC,0.9406,0.9398
4,KNeighborsClassifier,0.9587,0.9582


In [7]:
## Ngram Walks
rdf2vec = RDF2VecTransformer(Word2Vec(workers=2, epochs=20), 
        walkers=[
        NGramWalker(
            2,
            None,
            n_jobs=4,
            sampler=UniformSampler(),
            random_state=RANDOM_STATE,
            md5_bytes=None,
        )
    ],
verbose=1
)
embeddings, literals = rdf2vec.fit_transform(kg,  np.array(entities))

100%|██████████| 37554/37554 [00:14<00:00, 2581.05it/s]


Extracted 354105 walks for 37554 entities (69.0343s)
Fitted 354105 walks (36.6385s)


In [8]:
train_embeddings = np.array(embeddings[:len(train_entities)])
test_embeddings = np.array(embeddings[len(train_entities):])

scores = get_scores(train_embeddings, test_embeddings, train_labels, test_labels)
scores
# clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
# models,predictions = clf.fit(train_embeddings, test_embeddings, train_labels, test_labels)
# models

,Model,Accuracy,F1 Macro
0,XGBClassifier,0.9182,0.9168
1,LogisticRegression,0.8482,0.8461
2,RandomForestClassifier,0.8941,0.8924
3,SVC,0.8951,0.8932
4,KNeighborsClassifier,0.9027,0.9010


In [9]:
# # Community walks
# rdf2vec = RDF2VecTransformer(Word2Vec(workers=2, epochs=20), 
#         walkers=[
#         CommunityWalker(
#             2,
#             None,
#             n_jobs=4,
#             sampler=UniformSampler(),
#             random_state=RANDOM_STATE,
#             md5_bytes=None,
#         )
#     ],
# verbose=1
# )
# embeddings, literals = rdf2vec.fit_transform(kg,  np.array(entities))

In [10]:
# train_embeddings = np.array(embeddings[:len(train_entities)])
# test_embeddings = np.array(embeddings[len(train_entities):])

# clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
# models,predictions = clf.fit(train_embeddings, test_embeddings, train_labels, test_labels)
# models

In [11]:
## Random Walks
rdf2vec = RDF2VecTransformer(Word2Vec(workers=2, epochs=20), 
        walkers=[
        RandomWalker(
            2,
            None,
            n_jobs=4,
            sampler=UniformSampler(),
            random_state=RANDOM_STATE,
            md5_bytes=None,
        )
    ],
verbose=1
)
embeddings, literals = rdf2vec.fit_transform(kg,  np.array(entities))

100%|██████████| 37554/37554 [00:14<00:00, 2647.11it/s]


Extracted 354105 walks for 37554 entities (70.4497s)
Fitted 354105 walks (37.1648s)


In [12]:
train_embeddings = np.array(embeddings[:len(train_entities)])
test_embeddings = np.array(embeddings[len(train_entities):])

scores = get_scores(train_embeddings, test_embeddings, train_labels, test_labels)
scores
# clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
# models,predictions = clf.fit(train_embeddings, test_embeddings, train_labels, test_labels)
# models

,Model,Accuracy,F1 Macro
0,XGBClassifier,0.9583,0.9575
1,LogisticRegression,0.9296,0.9281
2,RandomForestClassifier,0.9478,0.9470
3,SVC,0.9494,0.9486
4,KNeighborsClassifier,0.9607,0.9599


In [13]:
# #WL walks
# rdf2vec = RDF2VecTransformer(Word2Vec(workers=2, epochs=20), 
#         walkers=[
#         WLWalker(
#             2,
#             None,
#             n_jobs=4,
#             sampler=UniformSampler(),
#             random_state=RANDOM_STATE,
#             md5_bytes=None,
#         )
#     ],
# verbose=1
# )
# embeddings, literals = rdf2vec.fit_transform(kg,  np.array(entities))

In [14]:
# train_embeddings = np.array(embeddings[:len(train_entities)])
# test_embeddings = np.array(embeddings[len(train_entities):])

# clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
# models,predictions = clf.fit(train_embeddings, test_embeddings, train_labels, test_labels)
# models

In [15]:
#Walklet walks
rdf2vec = RDF2VecTransformer(Word2Vec(workers=2, epochs=20), 
        walkers=[
        WalkletWalker(
            2,
            None,
            n_jobs=4,
            sampler=UniformSampler(),
            random_state=RANDOM_STATE,
            md5_bytes=None,
        )
    ],
verbose=1
)
embeddings, literals = rdf2vec.fit_transform(kg,  np.array(entities))

100%|██████████| 37554/37554 [00:11<00:00, 3151.51it/s]


Extracted 646530 walks for 37554 entities (62.8111s)
Fitted 646530 walks (41.2427s)


In [16]:
train_embeddings = np.array(embeddings[:len(train_entities)])
test_embeddings = np.array(embeddings[len(train_entities):])

scores = get_scores(train_embeddings, test_embeddings, train_labels, test_labels)
scores
# clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
# models,predictions = clf.fit(train_embeddings, test_embeddings, train_labels, test_labels)
# models

,Model,Accuracy,F1 Macro
0,XGBClassifier,0.9289,0.9281
1,LogisticRegression,0.8884,0.8864
2,RandomForestClassifier,0.9225,0.9213
3,SVC,0.9057,0.9044
4,KNeighborsClassifier,0.9189,0.9178
